<a href="https://colab.research.google.com/github/vvrgit/FDP-NNDL/blob/main/Time_Series_Forecasting_Using_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import math

In [3]:
load_data=pd.read_excel("/content/drive/MyDrive/NNDL-Data/Load Data in kW.xlsx")
load_data.head()

,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [4]:
dataset = load_data.drop(['Date','Time'], axis=1)
dataset.head()

,Load (kW)
0,5551.82208
1,4983.17184
2,4888.39680
3,5072.95872
4,5196.25980


In [5]:
# Setting 80 percent data for training
training_data_len = math.ceil(len(dataset['Load (kW)']) * .8)
training_data_len

1748

In [6]:
#Splitting the dataset
train_data = dataset[:training_data_len].iloc[:,:1]
test_data = dataset[training_data_len:].iloc[:,:1]
print(train_data.shape, test_data.shape)

(1748, 1) (436, 1)


In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# scaling dataset
scaled_train = scaler.fit_transform(train_data)
print(scaled_train[:5])

[[0.39751641]
 [0.29337711]
 [0.27602055]
 [0.30982015]
 [0.3324008 ]]


In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# scaling dataset
scaled_test = scaler.fit_transform(test_data)
print(scaled_train[:5])

[[0.39751641]
 [0.29337711]
 [0.27602055]
 [0.30982015]
 [0.3324008 ]]


In [9]:
X_train = []
y_train = []
for i in range(50, len(scaled_train)):
    X_train.append(scaled_train[i-50:i, 0])
    y_train.append(scaled_train[i, 0])
    if i <= 51:
        print(X_train)
        print(y_train)
        print()

[array([0.39751641, 0.29337711, 0.27602055, 0.30982015, 0.3324008 ,
       0.41390237, 0.69077933, 0.8183557 , 0.54002284, 0.53322866,
       0.36051955, 0.33773908, 0.29654582, 0.25535255, 0.51201827,
       0.51201827, 0.50810734, 0.51844134, 0.7404225 , 0.73733942,
       0.67522124, 0.57405081, 0.5437054 , 0.38843848, 0.28387097,
       0.25535255, 0.24404796, 0.23950899, 0.26802741, 0.40111333,
       0.50719383, 0.53385669, 0.57393663, 0.52897516, 0.40762204,
       0.46728518, 0.4201827 , 0.42012561, 0.37622038, 0.40134171,
       0.48384242, 0.49982872, 0.83702541, 0.84824436, 0.77111048,
       0.67887525, 0.56697117, 0.45186983, 0.3807879 , 0.31438767])]
[0.3079931487296602]

[array([0.39751641, 0.29337711, 0.27602055, 0.30982015, 0.3324008 ,
       0.41390237, 0.69077933, 0.8183557 , 0.54002284, 0.53322866,
       0.36051955, 0.33773908, 0.29654582, 0.25535255, 0.51201827,
       0.51201827, 0.50810734, 0.51844134, 0.7404225 , 0.73733942,
       0.67522124, 0.57405081, 0.543

In [10]:
X_test = []
y_test = []
for i in range(50, len(scaled_test)):
    X_test.append(scaled_test[i-50:i, 0])
    y_test.append(scaled_test[i, 0])

In [11]:
# The data is converted to Numpy array
X_train, y_train = np.array(X_train), np.array(y_train)

#Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
y_train = np.reshape(y_train, (y_train.shape[0],1))
print("X_train :",X_train.shape,"y_train :",y_train.shape)

X_train : (1698, 50, 1) y_train : (1698, 1)


In [12]:
# The data is converted to numpy array
X_test, y_test = np.array(X_test), np.array(y_test)

#Reshaping
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
y_test = np.reshape(y_test, (y_test.shape[0],1))
print("X_test :",X_test.shape,"y_test :",y_test.shape)

X_test : (386, 50, 1) y_test : (386, 1)


In [23]:
# importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout
from keras.layers import GRU, Bidirectional
from tensorflow.keras.optimizers import SGD
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import keras

In [18]:
# initializing the RNN
regressor = Sequential()

# adding RNN layers and dropout regularization
regressor.add(SimpleRNN(units = 50,
                        activation = "tanh",
                        return_sequences = True,
                        input_shape = (X_train.shape[1],1)))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units = 50,
                        activation = "tanh",
                        return_sequences = True))

regressor.add(SimpleRNN(units = 50,
                        activation = "tanh",
                        return_sequences = True))

regressor.add(SimpleRNN(units = 50,
                        activation = "tanh",
                        return_sequences = True))

# adding the output layer
regressor.add(Dense(units = 1,activation='sigmoid'))
regressor.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_12 (SimpleRNN)   (None, 50, 50)            2600      
                                                                 
 dropout_3 (Dropout)         (None, 50, 50)            0         
                                                                 
 simple_rnn_13 (SimpleRNN)   (None, 50, 50)            5050      
                                                                 
 simple_rnn_14 (SimpleRNN)   (None, 50, 50)            5050      
                                                                 
 simple_rnn_15 (SimpleRNN)   (None, 50, 50)            5050      
                                                                 
 dense_3 (Dense)             (None, 50, 1)             51        
                                                                 
Total params: 17801 (69.54 KB)
Trainable params: 17801

In [19]:
regressor.compile(loss='MSE',
              optimizer=SGD(),
              metrics=['MSE'])

In [24]:
checkpoint_filepath = 'RNN.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True)

In [25]:
# Write the Training input and output variables, size of the batch, number of epochs
history = regressor.fit(X_train,y_train,validation_data=(X_test, y_test),
                    batch_size=32,
                    epochs=1,verbose=1,callbacks=[model_checkpoint_callback])

54/54 [==============================] - 58s 70ms/step - loss: 0.0253 - MSE: 0.0253 - val_loss: 0.0307 - val_MSE: 0.0307


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
# Write the testing input and output variables
score = regressor.evaluate(X_test, y_test, verbose=2)
print('Test loss:', score[0])

13/13 - 0s - loss: 0.0307 - MSE: 0.0307 - 224ms/epoch - 17ms/step
Test loss: 0.030697770416736603


In [27]:
# load model
from keras.models import load_model
model = load_model('RNN.h5')
# summarize model.
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_12 (SimpleRNN)   (None, 50, 50)            2600      
                                                                 
 dropout_3 (Dropout)         (None, 50, 50)            0         
                                                                 
 simple_rnn_13 (SimpleRNN)   (None, 50, 50)            5050      
                                                                 
 simple_rnn_14 (SimpleRNN)   (None, 50, 50)            5050      
                                                                 
 simple_rnn_15 (SimpleRNN)   (None, 50, 50)            5050      
                                                                 
 dense_3 (Dense)             (None, 50, 1)             51        
                                                                 
Total params: 17801 (69.54 KB)
Trainable params: 17801